In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
from iuvs import io

In [57]:
darks = io.l1a_filenames('-?uvdark')

In [3]:
from IPython.parallel import Client
c = Client()

In [4]:
dview = c.direct_view()
lview = c.load_balanced_view()

In [12]:
%%px
from iuvs import io
import pandas as pd

In [30]:
def process_fname(fname):
    import pandas as pd
    from iuvs import io
    import numpy as np
    l1a = io.L1AReader(fname)
    header_df = io.get_header_df(l1a.hdulist[0]).T.convert_objects()
    if not hasattr(header_df, 'NAXIS3'):
        header_df['NAXIS3'] = np.nan
    header_df.drop(['XUV', 'OBS_ID', 'INT_TIME'], axis=1, inplace=True)
    df = pd.concat([header_df,
                    l1a.Engineering.T.convert_objects(),],
                   axis=1)
    df['dark_mean'] = l1a.img.mean()
    if df.size != 61:
        return (df, False)
    else:
        savepath = io.save_to_hdf(df, fname, 'l1a_dark_scans')
        return (df, True)

In [42]:
results = lview.map_async(process_fname, darks)

In [44]:
import time
while not results.ready():
    print(100*results.progress/len(darks))
    time.sleep(10)

91.94331197825665
93.57406328868181
95.5154338963308
97.25296059017667
98.67016113376043
99.6990875558144


In [1]:
import pandas as pd

In [2]:
import glob
h5fnames = glob.glob("/home/klay6683/output/l1a_dark_scans/*.h5")

In [3]:
len(h5fnames)

9769

In [18]:
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

In [36]:
dfs = []
for i,chunk in enumerate(chunker(h5fnames, 200)):
    print("Chunk {}".format(i))
    frames = []
    for fname in chunk:
        frames.append(pd.read_hdf(fname, 'df'))
    dfs.append(pd.concat(frames, ignore_index=True))

Chunk 0
Chunk 1
Chunk 2
Chunk 3
Chunk 4
Chunk 5
Chunk 6
Chunk 7
Chunk 8
Chunk 9
Chunk 10
Chunk 11
Chunk 12
Chunk 13
Chunk 14
Chunk 15
Chunk 16
Chunk 17
Chunk 18
Chunk 19
Chunk 20
Chunk 21
Chunk 22
Chunk 23
Chunk 24
Chunk 25
Chunk 26
Chunk 27
Chunk 28
Chunk 29
Chunk 30
Chunk 31
Chunk 32
Chunk 33
Chunk 34
Chunk 35
Chunk 36
Chunk 37
Chunk 38
Chunk 39
Chunk 40
Chunk 41
Chunk 42
Chunk 43
Chunk 44
Chunk 45
Chunk 46
Chunk 47
Chunk 48


In [37]:
superdf = pd.concat(dfs, ignore_index=True)

In [38]:
superdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9769 entries, 0 to 9768
Data columns (total 61 columns):
AVERAGE               9769 non-null float64
BIN_SHIFT             9769 non-null int64
BIN_SHIFT_DIR         9769 non-null int64
BIN_TBL               9769 non-null object
BIN_TYPE              9769 non-null object
BIN_X_ROW             9769 non-null int64
BIN_Y_ROW             9769 non-null int64
BITPIX                9769 non-null int64
BLANK                 9769 non-null int64
CADENCE               9769 non-null float64
CAPTURE               9769 non-null object
CASE_TEMP             9769 non-null float64
CHECKSUM              9769 non-null float64
DATA_COMPRESSION      9769 non-null int64
DET_TEMP              9769 non-null float64
EXTEND                9769 non-null bool
FILENAME              9769 non-null object
FUV_BAD_PIXEL_MASK    9769 non-null int64
IMAGE_NUMBER          9769 non-null float64
INT_TIME              9769 non-null float64
LENGTH                9769 non-null 

In [39]:
superdf.to_hdf('/home/klay6683/output/l1a_dark_stats.h5','df')